<a href="https://colab.research.google.com/github/gyeongseoMin/DataScience/blob/main/%EB%AF%BC%EA%B2%BD%EC%84%9C_20203060_%EA%B3%BC%EC%A0%9C9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import requests
import re
from collections import Counter
import math
import random
r = requests.get("http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
with open("sms.zip", "wb") as f:
     f.write(r.content)

zipfile.ZipFile("sms.zip").extractall("./")

In [ ]:
data = []
with open("SMSSpamCollection.txt", "r") as f:
    for line in f:
        data.append(line.strip().split("\t"))

#data
for d in data:
    d[1] = set(re.findall("[a-z0-1_]+", d[1].lower()))
    
random.shuffle(data)

train_len = int(len(data) * 0.8)
test_len = len(data) - train_len

train = data[:train_len]
test = data[train_len:]


In [ ]:
# bag-of-words 는 단어집합(중복되는 것들은 제거)
# 텍스트를 소문자나 대문자로 바꾸기
# regualr expression
# text patern 찾는 .. 
# text.lower() - 소문자로 바꾸기
# re.findall('[a-z0-9_+', txt.lower()) 


In [ ]:
train_spam = [bow for cls, bow in train if cls == 'spam']
train_ham = [bow for cls, bow in train if cls == 'ham']

In [ ]:
prior_spam = math.log(len(train_spam) / len(train))
prior_ham = math.log(len(train_ham) / len(train))
print(prior_spam)
print(prior_ham)

-2.0057501494113303
-0.14451644308985795


In [ ]:
# 각 단어의 가능도(likelihood) 구하기
# 단어 w의 spam 에서의 가능도 P(w | spam) = (w를 포함하는 스팸 수 + alpha) / (전체 스팸 수 + alpha)
# 단어 w의 ham에서의 가능도 P(w | ham) = (w를 포함하는 햄 수 + alpha) / (전체 햄 수  + alpha)
# 전체 스팸 수 = n_spam
# w를 포함하는 스팸 수, 햄 수
spam_words = Counter(word for bow in train_spam for word in bow)
ham_words = Counter(word for bow in train_ham for word in bow)
print(spam_words)
print(ham_words)

Counter({'0': 445, '1': 391, 'to': 376, 'call': 260, 'a': 241, 'you': 189, 'your': 180, '00': 172, 'now': 157, 'or': 147, 'for': 146, 'free': 142, 'the': 133, 'txt': 123, 'u': 114, 'is': 111, 'have': 110, '000': 103, 'on': 101, 'from': 95, '0p': 95, 'mobile': 93, 'ur': 93, 'text': 89, 'claim': 88, 'and': 86, 'stop': 82, 'with': 81, 'reply': 78, 't': 78, 'of': 77, 'this': 74, 'www': 71, 'our': 70, 'get': 70, 'only': 70, 'won': 66, 's': 65, 'prize': 64, 'just': 63, 'are': 63, 'in': 59, 'send': 56, '01': 55, 'new': 55, 'cash': 54, 'uk': 53, 'win': 51, 'we': 51, 'no': 50, 'contact': 48, 'please': 47, 'c': 46, 'service': 45, 'be': 44, 'out': 44, 'box': 44, 'urgent': 44, 'com': 43, '100': 43, 'msg': 42, '10': 42, 'co': 41, 'guaranteed': 40, 'nokia': 39, 'customer': 38, 'i': 37, 'per': 36, 'm': 36, 'draw': 36, 'week': 36, '1000': 36, 'will': 34, 'cs': 34, 'phone': 33, 'been': 33, '11': 32, 'mins': 32, 'tone': 31, 'min': 31, '0ppm': 30, 'pobox': 29, 'sms': 29, 'latest': 29, 'chat': 28, 'awarde

In [ ]:
def predict(bow, spam_words, ham_words, n_spams, n_hams, alpha):
    spam_score = prior_spam + sum(math.log((spam_words[word] +alpha) / (n_spams + alpha)) for word in bow)
    ham_score = prior_ham + sum(math.log((ham_words[word] + alpha) / (n_hams + alpha)) for word in bow)

    if(spam_score < ham_score):
        return 'ham'
    else:
        return 'spam'

In [ ]:
tp, tn, fp, fn = 0, 0, 0, 0
for cls, bow in test:
    pred = predict(bow, spam_words, ham_words, len(train_spam), len(train_ham), 1e-4)
    if cls == 'spam' and pred == 'spam': tp += 1
    elif cls == 'ham' and pred == 'ham': tn += 1
    elif cls == 'ham' and pred == 'spam':fp += 1
    else: fn += 1

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp) 
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
print(accuracy, precision, recall, f1)

0.9623318385650225 0.7837837837837838 0.9863945578231292 0.8734939759036144
